In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#set to: true - to read results for fibre stiffness model run
#         false - to read results for truss model run
fibre_stiffness = True

#operators_list = ['Remove_intersection','One_point_crossover_and_bitflip_mutation','Add_member']

#### Defining the functions
def read_csv_file(fib_stiff,run_num):
    filepath = 'C:\\SEAK Lab\\SEAK Lab Github\\KD3M3\\Truss_AOS\\result\\AOS MOEA Runs\\'

    if fib_stiff:
        filename = 'Fibre Stiffness code run results\\Feas and Stab False\\constraint_adaptive' + str(run_num) + '_qual.csv'
    else: 
        filename = 'Truss code run results\\Feas and Stab False\\constraint_adaptive' + str(run_num) + '_qual.csv' 

    full_filepath = filepath + filename

    with open(full_filepath,newline='') as csvfile:
        data = [row for row in csv.reader(csvfile)]
        if (data[1][0] == '.RemoveIntersection'):
            iter_num_operator1 = list(map(float,data[0][1:]))
            quality_operator1 = list(map(float,data[1][1:]))
            if (data[3][0] == 'OnePointCrossover+BitFlip'):
                iter_num_operator2 = list(map(float,data[2][1:]))
                quality_operator2 = list(map(float,data[3][1:]))
                iter_num_operator3 = list(map(float,data[4][1:]))
                quality_operator3 = list(map(float,data[5][1:]))
            elif (data[3][0] == '.AddMember'):
                iter_num_operator2 = list(map(float,data[4][1:]))
                quality_operator2 = list(map(float,data[5][1:]))
                iter_num_operator3 = list(map(float,data[2][1:]))
                quality_operator3 = list(map(float,data[3][1:]))
        elif (data[1][0] == '.AddMember'):
            iter_num_operator3 = list(map(float,data[0][1:]))
            quality_operator3 = list(map(float,data[1][1:]))
            if (data[3][0] == '.RemoveIntersection'):
                iter_num_operator1 = list(map(float,data[2][1:]))
                quality_operator1 = list(map(float,data[3][1:]))
                iter_num_operator2 = list(map(float,data[4][1:]))
                quality_operator2 = list(map(float,data[5][1:]))
            elif (data[3][0] == 'OnePointCrossover+BitFlip'):
                iter_num_operator2 = list(map(float,data[2][1:]))
                quality_operator2 = list(map(float,data[3][1:]))
                iter_num_operator1 = list(map(float,data[4][1:]))
                quality_operator1 = list(map(float,data[5][1:]))
        elif (data[1][0] == 'OnePointCrossover+BitFlip'):
            iter_num_operator2 = list(map(float,data[0][1:]))
            quality_operator2 = list(map(float,data[1][1:]))
            if (data[3][0] == '.RemoveIntersection'):
                iter_num_operator1 = list(map(float,data[2][1:]))
                quality_operator1 = list(map(float,data[3][1:]))
                iter_num_operator3 = list(map(float,data[4][1:]))
                quality_operator3 = list(map(float,data[5][1:]))
            elif (data[3][0] == '.AddMember'):
                iter_num_operator3 = list(map(float,data[2][1:]))
                quality_operator3 = list(map(float,data[3][1:]))
                iter_num_operator1 = list(map(float,data[4][1:]))
                quality_operator1 = list(map(float,data[5][1:]))
        
    return iter_num_operator1, quality_operator1, iter_num_operator2, quality_operator2, iter_num_operator3, quality_operator3

def create_full_quality_arrays(num_iter_op1, quality_op1, num_iter_op2, quality_op2, num_iter_op3, quality_op3):
    quality_full_operator1 = np.zeros(9900)
    quality_full_operator2 = np.zeros(9900)
    quality_full_operator3 = np.zeros(9900)
    index_count = 0
    for i in range(100,10000):
        if i in num_iter_op1:
            quality_full_operator1[index_count] = quality_op1[num_iter_op1.index(i)]
        elif i in num_iter_op2:
            quality_full_operator2[index_count] = quality_op2[num_iter_op2.index(i)]
        elif i in num_iter_op3:
            quality_full_operator3[index_count] = quality_op3[num_iter_op3.index(i)]
        index_count += 1
    
    iter_num_array = np.arange(100,10000,1)
    
    return iter_num_array, quality_full_operator1, quality_full_operator2, quality_full_operator3 

In [ ]:
#### Program operation
num_runs = 30
quality_op1_dict = {new_list: [] for new_list in range(num_runs)}
quality_op2_dict = {new_list: [] for new_list in range(num_runs)}
quality_op3_dict = {new_list: [] for new_list in range(num_runs)}

for i in range(num_runs):
    print('Reading and creating full quality arrays for run ' + str(i))
    n_iter_op1, qual_op1, n_iter_op2, qual_op2, n_iter_op3, qual_op3 = read_csv_file(fibre_stiffness,i)
    n_iter_array, qual_full_op1, qual_full_op2, qual_full_op3 = create_full_quality_arrays(n_iter_op1, qual_op1, n_iter_op2, qual_op2, n_iter_op3, qual_op3)
    quality_op1_dict[i] = qual_full_op1
    quality_op2_dict[i] = qual_full_op2
    quality_op3_dict[i] = qual_full_op3
    
## Compute the averaged credits for each operator over the runs
n_datapoints = len(quality_op1_dict[0])
quality_opt1_vals = np.zeros(num_runs)  
quality_opt2_vals = np.zeros(num_runs)
quality_opt3_vals = np.zeros(num_runs)

quality_op1_mean = np.zeros(n_datapoints)
quality_op2_mean = np.zeros(n_datapoints)
quality_op3_mean = np.zeros(n_datapoints)
quality_op1_std = np.zeros(n_datapoints)
quality_op2_std = np.zeros(n_datapoints)
quality_op3_std = np.zeros(n_datapoints)

print('Computing averaged credit values for each operator')
for i in range(n_datapoints):
    for j in range(num_runs):
        quality_opt1_current = quality_op1_dict[j]
        quality_opt2_current = quality_op2_dict[j]
        quality_opt3_current = quality_op3_dict[j]
        quality_opt1_vals[j] = quality_opt1_current[i]
        quality_opt2_vals[j] = quality_opt2_current[i]
        quality_opt3_vals[j] = quality_opt3_current[i]
    quality_op1_mean[i] = np.mean(quality_opt1_vals)
    quality_op2_mean[i] = np.mean(quality_opt2_vals)
    quality_op3_mean[i] = np.mean(quality_opt3_vals)
    quality_op1_std[i] = np.std(quality_opt1_vals)
    quality_op2_std[i] = np.std(quality_opt2_vals)
    quality_op3_std[i] = np.std(quality_opt3_vals)
    
    
## Plotting
print('Plotting')    
plt.figure()
plt.plot(n_iter_array, quality_op1_mean, 'b-', label='Remove Intersection')
plt.plot(n_iter_array, quality_op1_mean + 2*quality_op1_std, 'b.')
plt.plot(n_iter_array, quality_op1_mean - 2*quality_op1_std, 'b.')
plt.plot(n_iter_array, quality_op2_mean, 'r-', label='Crossover + Mutation')
plt.plot(n_iter_array, quality_op2_mean + 2*quality_op2_std, 'r.')
plt.plot(n_iter_array, quality_op2_mean - 2*quality_op2_std, 'r.')
plt.plot(n_iter_array, quality_op3_mean, 'g-', label='Add Member')
plt.plot(n_iter_array, quality_op3_mean + 2*quality_op3_std, 'g.')
plt.plot(n_iter_array, quality_op3_mean - 2*quality_op3_std, 'g.')
plt.xlabel('numbet of function evaluations')
plt.ylabel('average operator credits over ' + str(num_runs) + ' runs')
plt.legend(loc='upper right')
plt.show()